# Table of Contents
 <p><div class="lev1"><a href="# Step 1. Process data"><span class="toc-item-num">1&nbsp;&nbsp;</span>TODO: Following given notebooks in HW1, create table of contents here :) </a></div>

This Jupyter extension might help. https://github.com/minrk/ipython_extensions

# Homework 03 - Interactive Viz

Build a Choropleth map which shows intuitively (i.e., use colors wisely) how much grant money goes to each Swiss canton. 

The P3 database of the SNSF (Swiss National Science Foundation) http://p3.snf.ch/Pages/DataAndDocumentation.aspx can be linked via the project number (Project Number) field, in order to create a relationship between people and projects or publications and projects. Raw data of this project:
- P3_GrantExport.csv
- P3_GrantExport_with_abstracts.csv
- P3_PersonExport.csv
- P3_PublicationExport.csv

# Overview

**TODO**  
The objective of this homework is to ...  
To achieve this goal, we ...

In [1]:
import requests
import folium
import pandas as pd
import numpy as np
import json
import random
import pprint

# Data preprocessing/cleaning/wrangling/?
First, we obtain information about grant records, and cantons in Switzerland.

## Grant data

We download the data file from P3 website. We try to clean up data and obtain all grant records.

In [2]:
GRANTS_FILE = 'data/P3_GrantExport.csv'

csv_data = pd.read_csv(GRANTS_FILE, sep = ';')
csv_data.dtypes

﻿"Project Number"                int64
Project Title                   object
Project Title English           object
Responsible Applicant           object
Funding Instrument              object
Funding Instrument Hierarchy    object
Institution                     object
University                      object
Discipline Number                int64
Discipline Name                 object
Discipline Name Hierarchy       object
Start Date                      object
End Date                        object
Approved Amount                 object
Keywords                        object
dtype: object

In [3]:
csv_data.head()

,"﻿""Project Number""",Project Title,Project Title English,Responsible Applicant,Funding Instrument,Funding Instrument Hierarchy,Institution,University,Discipline Number,Discipline Name,Discipline Name Hierarchy,Start Date,End Date,Approved Amount,Keywords
0,1,Schlussband (Bd. VI) der Jacob Burckhardt-Biog...,NaN,Kaegi Werner,Project funding (Div. I-III),Project funding,NaN,Nicht zuteilbar - NA,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,11619.00,NaN
1,4,Batterie de tests à l'usage des enseignants po...,NaN,Massarenti Léonard,Project funding (Div. I-III),Project funding,Faculté de Psychologie et des Sciences de l'Ed...,Université de Genève - GE,10104,Educational science and Pedagogy,"Human and Social Sciences;Psychology, educatio...",01.10.1975,30.09.1976,41022.00,NaN
2,5,"Kritische Erstausgabe der ""Evidentiae contra D...",NaN,Kommission für das Corpus philosophorum medii ...,Project funding (Div. I-III),Project funding,Kommission für das Corpus philosophorum medii ...,"NPO (Biblioth., Museen, Verwalt.) - NPO",10101,Philosophy,Human and Social Sciences;Linguistics and lite...,01.03.1976,28.02.1985,79732.00,NaN
3,6,Katalog der datierten Handschriften in der Sch...,NaN,Burckhardt Max,Project funding (Div. I-III),Project funding,Abt. Handschriften und Alte Drucke Bibliothek ...,Universität Basel - BS,10302,Swiss history,Human and Social Sciences;Theology & religious...,01.10.1975,30.09.1976,52627.00,NaN
4,7,Wissenschaftliche Mitarbeit am Thesaurus Lingu...,NaN,Schweiz. Thesauruskommission,Project funding (Div. I-III),Project funding,Schweiz. Thesauruskommission,"NPO (Biblioth., Museen, Verwalt.) - NPO",10303,Ancient history and Classical studies,Human and Social Sciences;Theology & religious...,01.01.1976,30.04.1978,120042.00,NaN


After further inspection, we summarize the data fields as follows:
* ** TODO **: add descriptions of all fields
* 

For this assignment, to compute the total amount of grants, we plan to find the canton of every record of grants, and group these amount together by canton. We note that information about university and institution can provide us information about the place of project. This information is crucial for us to identify its canton.

Thus, we are interested in following three data fields :
* ** Institution **: the institution where the project is carried.
* ** University **: the university where the project is carried.
* ** Approved Amount **: amount of approved grants

We extract these columns from initial data to a separate DataFrame.

In [4]:
data = csv_data[['University', 'Institution', 'Approved Amount']]
data.head()

,University,Institution,Approved Amount
0,Nicht zuteilbar - NA,NaN,11619.00
1,Université de Genève - GE,Faculté de Psychologie et des Sciences de l'Ed...,41022.00
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",Kommission für das Corpus philosophorum medii ...,79732.00
3,Universität Basel - BS,Abt. Handschriften und Alte Drucke Bibliothek ...,52627.00
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",Schweiz. Thesauruskommission,120042.00


Next, we clean up the data.  
We note that some values of 'Approved Amount' field are not numeric. Instead, they are strings indicating the lack of data. So, we only keep records with numberic values.

In [5]:
data['Approved Amount'] = pd.to_numeric(data['Approved Amount'], errors='coerce')

c:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


We also remove records with illegal universities and null approved amount

In [6]:
data = data[(data.University != 'Nicht zuteilbar - NA') & pd.notnull(data.University) & pd.notnull(data['Approved Amount'])]

In [7]:
data.head()

,University,Institution,Approved Amount
1,Université de Genève - GE,Faculté de Psychologie et des Sciences de l'Ed...,41022.0
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",Kommission für das Corpus philosophorum medii ...,79732.0
3,Universität Basel - BS,Abt. Handschriften und Alte Drucke Bibliothek ...,52627.0
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",Schweiz. Thesauruskommission,120042.0
5,Université de Fribourg - FR,"Séminaire de politique économique, d'économie ...",53009.0


In [8]:
# TODO. It's a temporary workaround.
# In next section, we need to split all university names by '-' to two parts: full name and short name.
# If there is no '-' in the string, extracting short name will get an error 
data['University'].replace("Forschungskommission SAGW", "Forschungskommission - SAGE", inplace=True)
universities = pd.DataFrame(data.University.unique())

uni_test = universities.replace({r'\s+': '&'}, regex=True)
uni_name = pd.DataFrame(universities[0].str.rsplit('-',1).tolist(), columns=['Full Name', 'Short Name'])
uni_name.head()

,Full Name,Short Name
0,Université de Genève,GE
1,"NPO (Biblioth., Museen, Verwalt.)",NPO
2,Universität Basel,BS
3,Université de Fribourg,FR
4,Universität Zürich,ZH


## Canton data

We obtain abbrevations for all cantons from geographical JSON data.
We did some inspection of the structure of the JSON file, and extract canton abbrevations from the specified label.

In [9]:
CH_GEO_JSON = 'data/ch-cantons.topojson.json'

with open(CH_GEO_JSON, 'r') as f:
    d = json.load(f)

cantons = d['objects']['cantons']['geometries']
canton_ids = []
for canton in cantons:
    canton_ids.append(canton['id'])

canton_ids

['ZH',
 'BE',
 'LU',
 'UR',
 'SZ',
 'OW',
 'NW',
 'GL',
 'ZG',
 'FR',
 'SO',
 'BS',
 'BL',
 'SH',
 'AR',
 'AI',
 'SG',
 'GR',
 'AG',
 'TG',
 'TI',
 'VD',
 'VS',
 'NE',
 'GE',
 'JU']

# Mapping to canton

To compute grants by canton, we need to map (University, Institution) to canton.

Available APIs to obtain such mappings include **Geonames Full Text Search API in JSON**, and **GoogleMaps API**, etc. 

## Geonames API

In [10]:
uni_canton = pd.DataFrame(columns=['University','Canton'])

In [11]:
def canton_in_CH(result):
    for i in range(len(result)):
        if(result['geonames'][i]['countryName']=='Switzerland'):
            return True
    return False

In [12]:
URL = 'http://api.geonames.org/searchJSON?'

def canton_get(uni):
    paraload = {'q':uni,'username':'shiyuenie'}
    r = requests.get(URL, params=paraload)
    result = r.json()
    return result

In [13]:
for ucount in range(len(universities)): # search full name
    uni = universities[0][ucount]
    result = canton_get(uni)
    if (result['geonames'] != []):
        if (canton_in_CH(result)):
            uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
        else:
            uni_canton.loc[ucount] = [uni,'Not Found']   
    else:
        uni_canton.loc[ucount] = [uni,'Not Found']
    
uni_canton.head()

,University,Canton
0,Université de Genève - GE,Not Found
1,"NPO (Biblioth., Museen, Verwalt.) - NPO",Not Found
2,Universität Basel - BS,Basel-City
3,Université de Fribourg - FR,Fribourg
4,Universität Zürich - ZH,Zurich


In [14]:
#uni_name=uni_name.replace({r'\s+': '&'}, regex=True)

for ucount in range(len(uni_name)):
    if (uni_canton.loc[:,'Canton'][ucount] == 'Not Found'): # search with seperate parts
        uni_full = uni_name['Full Name'][ucount] 
        uni_short = uni_name['Short Name'][ucount] 
        result = canton_get(uni_full) # search part 1
        if (result['geonames'] != []):
            if(canton_in_CH(result)):
                uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
            else:
                result = canton_get(uni_short) # search part 2 
                if (result['geonames'] != []):
                    if(canton_in_CH(result)):
                        uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
        
uni_canton.head()

,University,Canton
0,Université de Genève - GE,Not Found
1,"NPO (Biblioth., Museen, Verwalt.) - NPO",Not Found
2,Universität Basel - BS,Basel-City
3,Université de Fribourg - FR,Fribourg
4,Universität Zürich - ZH,Zurich


In [15]:
uni_canton[uni_canton['Canton'] != "Not Found"]

for ucount in range(len(uni_test)): # search full name wit &
    uni = uni_test[0][ucount]
    result = canton_get(uni)
    if (result['geonames'] != []):
        if (canton_in_CH(result)):
            uni_canton.loc[ucount] = [uni,result['geonames'][0]['adminName1']]
    
uni_canton.head()

,University,Canton
0,Université de Genève - GE,Not Found
1,"NPO (Biblioth., Museen, Verwalt.) - NPO",Not Found
2,Universität Basel - BS,Basel-City
3,Université de Fribourg - FR,Fribourg
4,Universität Zürich - ZH,Zurich


In [16]:
np.sum(uni_canton.loc[:,'Canton']=='Not Found')

68

In [17]:
uni_canton[uni_canton['Canton'] != "Not Found"]

,University,Canton
2,Universität Basel - BS,Basel-City
3,Université de Fribourg - FR,Fribourg
4,Universität Zürich - ZH,Zurich
5,Fernfachhochschule Schweiz (Mitglied SUPSI) - ...,Vaud
6,Universität Bern - BE,Bern
8,Université de Neuchâtel - NE,Neuchâtel
9,Fernfachhochschule Schweiz (Mitglied SUPSI) - ...,Zurich
34,Fernfachhochschule Schweiz (Mitglied SUPSI) - ...,Saint Gallen


## GoogleMaps API

We use a Python [API wrapper](https://github.com/slimkrazy/python-google-places) for GoogleMaps

In [18]:
# to install the wrapper
# !pip install https://github.com/slimkrazy/python-google-places/zipball/master

In [19]:
import googlemaps

# Key omitted due to security concerns on Github
API_KEY = #########################'

gmaps = googlemaps.Client(key=API_KEY)

In [20]:
# We try a sample search to obtain the structure of reply
test_epfl = gmaps.geocode('EPFL Switzerland')
test_epfl

[{'address_components': [{'long_name': 'EPFL',
    'short_name': 'EPFL',
    'types': ['premise']},
   {'long_name': 'Lausanne',
    'short_name': 'Lausanne',
    'types': ['locality', 'political']},
   {'long_name': 'Ouest lausannois',
    'short_name': 'Ouest lausannois',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Vaud',
    'short_name': 'VD',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Switzerland',
    'short_name': 'CH',
    'types': ['country', 'political']},
   {'long_name': '1015', 'short_name': '1015', 'types': ['postal_code']}],
  'formatted_address': 'EPFL, 1015 Lausanne, Switzerland',
  'geometry': {'location': {'lat': 46.5189865, 'lng': 6.5676007},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 46.52033548029151,
     'lng': 6.568949680291502},
    'southwest': {'lat': 46.51763751970851, 'lng': 6.566251719708498}}},
  'place_id': 'ChIJ4zm3ev4wjEcRShTLf2C0UWA',
  'types': ['premise']}

We notice that canton information is returned in 'administrative_area_level_2' field. long_name gives the full name of the canton, short_name gives the abbrevation, the same as the id used in geometry json file.

In [21]:
CANTON_TYPE_KEY = 'administrative_area_level_1'
    
def place_in_CH(place_info):
    for info_dict in place_info['address_components']:
        if info_dict['long_name'] == 'Switzerland':
            return True
    return False

def parse_canton(place_candidates):
    if (place_candidates == None or len(place_candidates) == 0):
        return None
    
    for place in place_candidates:
        if not place_in_CH(place):
            continue
        for info_dict in place['address_components']:
            if (CANTON_TYPE_KEY in info_dict['types']) and (info_dict['short_name'] in canton_ids):
                return info_dict['short_name']
    
    return None    

parse_canton(test_epfl)

'VD'

In [22]:
def construct_search_keys(university, institution):
    
    def keys_with(x):
        keys = []
        keys.append(x)
        keys.append(x + ', Switzerland')
        keys.append(x + ', CH')
        return keys
    
    uni_full_name, uni_short_name = university.rsplit('-',1)

    keys = []   
    keys += keys_with(university)
    keys += keys_with(uni_full_name)
    
    if institution != None:
        keys += keys_with(str(institution))
    if uni_short_name != None:
        keys += keys_with(uni_short_name)
    return keys 

In [23]:
def search_canton(keys):
    for key in keys:
        resp = gmaps.geocode(key)
        canton = parse_canton(resp)
        if canton != None:
            return canton
    return None

In [24]:
# intialize uni_canton_dict from file
# uni_canton_dict maps university to canton
uni_canton_df = pd.read_csv('uni-canton.csv', encoding='utf-8')
uni_canton_df.set_index('University', inplace=True)
uni_canton_dict = uni_canton_df.to_dict()['0']

In [25]:
def find_canton(row):
    # if not searched before
    if row.University not in uni_canton_dict:
        search_keys = construct_search_keys(row.University, row.Institution)
        uni_canton_dict[row.University] = search_canton(search_keys)
        print(row.University + " " + uni_canton_dict[row.University])
    return uni_canton_dict[row.University]

In [26]:
# uni_canton_df = pd.DataFrame.from_dict(uni_canton_dict, orient="index")
# uni_canton_df.index.name = 'University'
# uni_canton_df.to_csv('uni-canton.csv', encoding='utf-8')

In [27]:
df = data.copy()
df['Canton'] = df.apply(find_canton, axis=1)

In [28]:
df.head()

,University,Institution,Approved Amount,Canton
1,Université de Genève - GE,Faculté de Psychologie et des Sciences de l'Ed...,41022.0,GE
2,"NPO (Biblioth., Museen, Verwalt.) - NPO",Kommission für das Corpus philosophorum medii ...,79732.0,FR
3,Universität Basel - BS,Abt. Handschriften und Alte Drucke Bibliothek ...,52627.0,BS
4,"NPO (Biblioth., Museen, Verwalt.) - NPO",Schweiz. Thesauruskommission,120042.0,FR
5,Université de Fribourg - FR,"Séminaire de politique économique, d'économie ...",53009.0,FR


In [29]:
df.to_csv('data-with-canton.csv', encoding='utf-8')

In [30]:
found_rate = pd.notnull(df['Canton']).sum() / len(df['Canton'])
found_rate

0.96396247126317747

### Manually handle missing data

Fow now, we simply ignore the missing records.

#  Compute grants by canton

In [31]:
canton_grants_df = df[['Approved Amount', 'Canton']].groupby(['Canton']).mean()
canton_grants_df.reset_index(inplace=True)

In [32]:
# Add missing data
for canton_id in canton_ids:
    if canton_id not in canton_grants_df['Canton'].values:
        canton_grants_df = canton_grants_df.append({'Canton':canton_id, 'Approved Amount': 0.0}, ignore_index=True)

canton_grants_df = canton_grants_df.sort_values(by=['Approved Amount'], ascending=False)
canton_grants_df

,Canton,Approved Amount
5,GE,288583.545632
13,TG,287070.071429
3,BS,285288.768854
16,VD,280711.058715
2,BE,276768.072614
19,ZH,266918.811070
14,TI,242504.534507
6,GR,234106.088595
4,FR,222939.102799
0,AG,215293.570682


# Data visualization
##  Draw grants map by canton

In [33]:
map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=canton_grants_df,
               columns=['Canton', 'Approved Amount'], 
               key_on='feature.id', 
               geo_path=CH_GEO_JSON, 
               topojson='objects.cantons', 
               fill_color='YlOrRd'
               )
map

c:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py:7: FutureWarning: 'threshold_scale' default behavior has changed. Now you get a linear scale between the 'min' and the 'max' of your data. To get former behavior, use folium.utilities.split_six.


** TODO **: The map cannot be displayed on Github. To view the map, you have to run this cell locally with Jupyter Notebook. To facilitate code reviewing, we may add an additionaly snapshot of the map here.